In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import joblib
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge
from skforecast.exceptions import OneStepAheadValidationWarning
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection import bayesian_search_forecaster_multiseries, grid_search_forecaster_multiseries
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold
from skforecast.preprocessing import RollingFeatures
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error

In [3]:
series_item_sales = pd.read_parquet(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_multi_series_items_sales.parquet")
series_item_sales = series_item_sales.asfreq('D')
exog_item_sales = pd.DataFrame({'day_of_week': series_item_sales.index.dayofweek}, index = series_item_sales.index)
series_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_sample_multi_series.joblib")
exog_dict = joblib.load(r"C:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\tests\fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
series_dict_train = {k: v.loc[:end_train,] for k, v in series_dict.items()}
exog_dict_train = {k: v.loc[:end_train,] for k, v in exog_dict.items()}
series_dict_test = {k: v.loc[end_train:,] for k, v in series_dict.items()}
exog_dict_test = {k: v.loc[end_train:,] for k, v in exog_dict.items()}

In [29]:
cv = OneStepAheadFold(initial_train_size = 100, return_all_indexes=True)
cv.split(y_feature_selection, as_pandas=False)

Information of folds
--------------------
Number of observations in train: 100
Number of observations in test: 400
Training : 2020-01-01 00:00:00 -- 2020-01-05 03:00:00 (n=100)
Test     : 2020-01-05 04:00:00 -- 2020-01-21 19:00:00 (n=400)



[[range(0, 100)], [range(100, 500)], True]

In [28]:
from skforecast.model_selection.tests.fixtures_model_selection import y_feature_selection

In [27]:
cv = TimeSeriesFold(initial_train_size = 1000, steps=97, return_all_indexes=True)
cv.split(series_item_sales, as_pandas=False)

Information of folds
--------------------
Number of observations used for initial training: 1000
Number of observations used for backtesting: 97
    Number of folds: 1
    Number skipped folds: 0 
    Number of steps per fold: 97
    Number of steps to exclude between last observed data (last window) and predictions (gap): 0

Fold: 0
    Training:   2012-01-01 00:00:00 -- 2014-09-26 00:00:00  (n=1000)
    Validation: 2014-09-27 00:00:00 -- 2015-01-01 00:00:00  (n=97)



c:\Users\jaesc2\GitHub\skforecast\skforecast\model_selection\_split.py:895: UserWarning: Last window cannot be calculated because `window_size` is None.
  warnings.warn(


[[range(0, 1000), None, range(1000, 1097), range(1000, 1097), True]]

In [11]:
metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
forecaster = ForecasterDirectMultiVariate(
                regressor          = Ridge(random_state=123),
                lags               = 10,
                steps              = 10,
                level              = 'item_1',
                transformer_series = StandardScaler(),
                transformer_exog   = StandardScaler(),
            )
cv = OneStepAheadFold(initial_train_size = pd.to_datetime('2014-09-26 00:00:00'))

def search_space(trial):
    search_space  = {
        'alpha': trial.suggest_float('alpha', 1e-3, 1e3, log=True),
        'lags': trial.suggest_categorical('lags', [3, 5]),
    }

    return search_space

warn_msg = re.escape(
    "One-step-ahead predictions are used for faster model comparison, but they "
    "may not fully represent multi-step prediction performance. It is recommended "
    "to backtest the final model for a more accurate multi-step performance "
    "estimate."
)
with pytest.warns(OneStepAheadValidationWarning, match = warn_msg):
    results, _ = bayesian_search_forecaster_multiseries(
        forecaster         = forecaster,
        series             = series_item_sales,
        exog               = exog_item_sales,
        cv                 = cv,
        search_space       = search_space,
        n_trials           = 5,
        metric             = metrics,
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )

expected_results = pd.DataFrame({
        "levels": [["item_1"], ["item_1"], ["item_1"], ["item_1"], ["item_1"]],
        "lags": [
            np.array([1, 2, 3, 4, 5]),
            np.array([1, 2, 3, 4, 5]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3]),
        ],
        "params": [
            {"alpha": 0.2252709077935534},
            {"alpha": 0.4279898484823994},
            {"alpha": 15.094374246471325},
            {"alpha": 2.031835829826598},
            {"alpha": 766.6289057556013},
        ],
        "mean_absolute_error": [
            0.8093780037271759,
            0.8094817020309184,
            0.8518429054223443,
            0.8539920097550104,
            1.0811971619067005,
        ],
        "mean_absolute_percentage_error": [
            0.03830937965446373,
            0.03831611554799997,
            0.0404554529541771,
            0.04045967644664634,
            0.05359275342361736,
        ],
        "mean_absolute_scaled_error": [
            0.5296430977017792,
            0.5297109560949745,
            0.5575335769867841,
            0.5589401718158097,
            0.7076465828014749,
        ],
        "alpha": [
            0.2252709077935534,
            0.4279898484823994,
            15.094374246471325,
            2.031835829826598,
            766.6289057556013,
        ],
    },
        index=pd.RangeIndex(start=0, stop=5, step=1)
    )

pd.testing.assert_frame_equal(results, expected_results)

In [13]:
param_grid = {'alpha': [0.01, 0.1, 1]}
lags_grid = [2, 4]

results = grid_search_forecaster_multiseries(
                  forecaster          = forecaster,
                  series              = series_dict,
                  param_grid          = param_grid,
                  cv                  = cv,
                  metric              = 'mean_absolute_error',
                  aggregate_metric    = 'weighted_average',
                  levels              = None,
                  exog                = None,
                  lags_grid           = lags_grid,
                  return_best         = False,
                  verbose             = True
              )

6 models compared for 5 level(s). Number of iterations: 6.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/3 [00:00<?, ?it/s]

UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U19'), dtype('int64')) -> None

In [19]:
from skforecast.utils import date_to_index_position

In [28]:
cv = OneStepAheadFold(
        initial_train_size = "2020-03-01 00:00:00",
)

In [29]:
n = 1000000

series = pd.Series(
    np.random.rand(n),
    index=pd.date_range(start='2020-01-01', periods=n, freq='h')
)

In [30]:
%%timeit

cv._extract_index(series)

171 ns ± 2.43 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [31]:
idx = cv._extract_index(series)

In [32]:
%%timeit

initial_train_size = date_to_index_position(
                                 index        = idx, 
                                 date_input   = cv.initial_train_size, 
                                 method       = 'validation',
                                 date_literal = 'initial_train_size'
                             )

326 μs ± 6.18 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [33]:
# Create a dict with 1000 pandas series
series_dict = {
    f'series_{i}': pd.Series(
        np.random.rand(n),
        index=pd.date_range(start='2020-01-01', periods=n, freq='h')
    )
    for i in range(1000)
}

In [34]:
%%timeit

cv._extract_index(series_dict)

10.3 ms ± 108 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
idx = cv._extract_index(series_dict)

In [36]:
%%timeit

initial_train_size = date_to_index_position(
                                 index        = idx, 
                                 date_input   = cv.initial_train_size, 
                                 method       = 'validation',
                                 date_literal = 'initial_train_size'
                             )

312 μs ± 5.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
import re
import os
import pytest
import numpy as np
import pandas as pd
import joblib
from pathlib import Path
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
from skforecast.exceptions import OneStepAheadValidationWarning
from skforecast.metrics import mean_absolute_scaled_error
from skforecast.metrics import root_mean_squared_scaled_error
from skforecast.preprocessing import RollingFeatures
from skforecast.recursive import ForecasterRecursiveMultiSeries
from skforecast.direct import ForecasterDirectMultiVariate
from skforecast.model_selection._search import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection._split import TimeSeriesFold, OneStepAheadFold

In [23]:
forecaster = ForecasterRecursiveMultiSeries(
                regressor=LGBMRegressor(random_state=678, verbose=-1),
                lags=3,
                transformer_series=None,
                forecaster_id='Multiseries_no_transformer'
            )

metrics = [
    "mean_absolute_error",
    "mean_squared_error",
    mean_absolute_percentage_error,
    mean_absolute_scaled_error,
    root_mean_squared_scaled_error,
]
param_grid = {
    "n_estimators": [5, 10],
    "max_depth": [2, 3]
}
lags_grid = [3, 5]
param_grid = list(ParameterGrid(param_grid))

cv_backtesting = TimeSeriesFold(
    initial_train_size = '2016-07-31 00:00:00',
    steps              = 1,
    refit              = False,
    fixed_train_size   = False,
)
cv_one_step_ahead = OneStepAheadFold(initial_train_size = '2016-07-31 00:00:00')

results_backtesting = _evaluate_grid_hyperparameters_multiseries(
    forecaster         = forecaster,
    series             = series_dict,
    exog               = exog_dict,
    cv                 = cv_backtesting,
    param_grid         = param_grid,
    lags_grid          = lags_grid,
    metric             = metrics,
    aggregate_metric   = ["average", "weighted_average", "pooling"],
    return_best        = False,
    n_jobs             = 'auto',
        suppress_warnings=True
)

warn_msg = re.escape(
    "One-step-ahead predictions are used for faster model comparison, but they "
    "may not fully represent multi-step prediction performance. It is recommended "
    "to backtest the final model for a more accurate multi-step performance "
    "estimate."
)
with pytest.warns(OneStepAheadValidationWarning, match = warn_msg):
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        cv                 = cv_one_step_ahead,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        n_jobs             = 'auto',
        suppress_warnings=True
    )

pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)

lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/4 [00:00<?, ?it/s]

c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\jaesc2\Miniconda3\envs\skforecast_py11_2\Lib\site-packages\sklearn\utils\deprecation.py:151: Future

lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/4 [00:00<?, ?it/s]

Failed: DID NOT WARN. No warnings of type (<class 'skforecast.exceptions.exceptions.OneStepAheadValidationWarning'>,) were emitted.
 Emitted warnings: [].